In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file_paths = {'in':'../data/enriched/Ride_to_alum_creek.csv',
              'out':'../data/enriched/Ride_to_alum_creek.csv'
             }

df = pd.read_csv(file_paths['in'])

In [3]:
df.dtypes

track             int64
segment           int64
time             object
time_utc        float64
time_diff       float64
elapsed_time    float64
elevation       float64
latitude        float64
longitude       float64
dtype: object

In [4]:
# convert the time column to a datetime type
df['time'] = pd.to_datetime(df['time'])

# 1. Calculating Distance Travelled

In [12]:
from haversine import haversine

In [32]:
def compute_distance(df, latitude='latitude', longitude='longitude'):
    df = df.copy()
    # Copy the previous values of Lat/Long to the current row for vectorized computation
    df['lat_old'] = df[latitude].shift()
    df['long_old'] = df[longitude].shift()
    
    # Grab the relevant columns for distance calculation
    df = df[['lat_old', 'long_old', latitude, longitude]]
    
    # Define an anonymous function to execute over each row to calculate the distance between rows
    haversine_distance = lambda x: haversine((x[0], x[1]), (x[2], x[3]), unit='mi')
    
    # Create the distance column, making sure to apply the function row-by-row
    df['distance'] = df.apply(haversine_distance, axis=1)
    return df['distance']
    

In [17]:
df = df.copy()
# Copy the previous values of Lat/Long to the current row for vectorized computation
df['lat_old'] = df['latitude'].shift()
df['long_old'] = df['longitude'].shift()

In [18]:
df_test = df[['lat_old', 'long_old', 'latitude', 'longitude']]

In [19]:
haversine_distance = lambda x: haversine((x[0], x[1]), (x[2], x[3]), unit='mi')

In [31]:
df_test.apply(haversine_distance, axis=1)

0            NaN
1       0.000556
2       0.000578
3       0.000578
4       0.000492
          ...   
8766    0.000069
8767    0.000173
8768    0.000214
8769    0.000138
8770    0.000069
Length: 8771, dtype: float64

In [33]:
compute_distance(df)

0            NaN
1       0.000556
2       0.000578
3       0.000578
4       0.000492
          ...   
8766    0.000069
8767    0.000173
8768    0.000214
8769    0.000138
8770    0.000069
Name: distance, Length: 8771, dtype: float64

In [7]:
df['longitude']

0      -83.036139
1      -83.036132
2      -83.036126
3      -83.036120
4      -83.036115
          ...    
8766   -83.036108
8767   -83.036105
8768   -83.036104
8769   -83.036104
8770   -83.036104
Name: longitude, Length: 8771, dtype: float64

In [8]:
df['longitude'].shift()

0             NaN
1      -83.036139
2      -83.036132
3      -83.036126
4      -83.036120
          ...    
8766   -83.036108
8767   -83.036108
8768   -83.036105
8769   -83.036104
8770   -83.036104
Name: longitude, Length: 8771, dtype: float64